# Imports

In [5]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split

# Data extraction into new Dataframe

In [6]:
# We don't need HomeID and AwayID here, but I put it in just for exploration
results = pd.read_pickle('preprocessed_results.pkl')
df = results[['HomeID', 'AwayID', 'FTHG', 'FTAG', 'FTR']].copy()
df

,HomeID,AwayID,FTHG,FTAG,FTR
0,0,15,3,0,H
1,1,12,1,0,H
2,2,16,0,0,D
3,3,14,6,0,H
4,4,11,2,2,D
...,...,...,...,...,...
13015,97,105,2,2,D
13016,111,106,2,0,H
13017,120,109,0,2,A
13018,122,107,2,2,D


In [10]:
results.sample(3)

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
1828,E1,2015-06-04,25,18,2,1,H,1,0,H,...,3,4,11,11,4,13,2,1,0,0
10497,SP1,2012-02-12,100,117,2,3,A,0,1,A,...,7,7,9,16,12,5,3,5,0,0
5139,D1,2016-02-14,38,41,3,2,H,2,1,H,...,9,4,15,9,3,6,3,1,0,0


In [7]:
# Replace H, D and A by 1, 0 and 2 (target variables)
result_dict = {'H': 1, 'D': 0, 'A': 2}
df['FTR'].replace(result_dict, inplace=True)

In [8]:
#slc = df['AwayID'] == 105
#df[slc][['FTR']].value_counts() / sum(slc)

In [9]:
# The columns H5GR and A5GR contain the points collected by the home
# and away teams over the last 5 games.
# (Win = 3 points, Draw = 1 point, Loss = 0 points)
# If there is not enough data, we just add 1.333333 points (or some mean maybe)
# until we get to 5 games.
df['H5GR'] = 0
df['A5GR'] = 0

cur_game = 0
for game in results.iterrows():
    team_id = game[1]['HomeID']
    team_points = 0         # points from the last 5 games
    
    recent_games = results.loc[(results['HomeID'] == team_id) | (results['AwayID'] == team_id)].tail(6) # TODO Date checken!
    recent_games.drop(recent_games.tail(1).index, inplace=True)   # drop cur_game from recent games
    
    # Add the points accordingly
    for recent in recent_games:
        if recent['FTR'] == 'D':
            team_points += 1
        elif (recent['FTR'] == '1' and recent['HomeID'] == team_id
            or recent['FTR'] == '2' and recent['AwayID'] == team_id):
            team_points += 3           
        # TODO funktioniert noch nicht

    
    df['H5GR'][cur_game] = team_points
    
    cur_game += 1


TypeError: string indices must be integers

In [ ]:
df.sample(3)